In [1]:
from typing import Iterable


def parser(file = 'input.txt'):
    with open(file, 'r') as file:
        # for line in file:
        #     yield line.strip('\n')
        workflows, cases = file.read().split('\n\n')
        workflows
        return workflows, cases

def solver(workflows, cases):
    ...
    return None


if __name__ == '__main__':
    test_p1 = solver(*parser('test_input.txt'))
    print(test_p1)
    assert test_p1 == None, test_p1
    # print(solver(parser()))

    # test_p2 = solver(parser('test_input.txt'))
    # test_p2 = solver(parser('test_input_p2.txt'))
    # print(test_p2)
    # assert test_p2 == None, test_p2
    # print(solver(parser()))

None
